# Download CAMELS-US shapefiles

In [1]:
import sys
import shutil
import requests
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [12]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [14]:
# Get the required info from the config file
data_path = cs.read_from_config(config_file,'data_path')
shps_path = cs.read_from_config(config_file,'ref_shps_path')
file_url  = cs.read_from_config(config_file,'us_camels_url')

### Make output folder

In [4]:
# Construct the download location
download_folder = Path(data_path) / shps_path / 'CAMELS-US'

In [5]:
# Make sure the download folder exists
download_folder.mkdir(parents=True, exist_ok=True)

### Get the data
Note: apparently there is no clean way to check if a file has been downloaded correctly without comparing hash strings, so we'll just assume that it downloads correctly.

In [6]:
# Retry settings
retries_max = 10

In [7]:
# Make the file name
file_name = file_url.split('/')[-1].strip() # Get the last part of the url, strip whitespace and characters

In [11]:
# Make sure the connection is re-tried if it fails
retries_cur = 1
while retries_cur <= retries_max:
    try: 

        # Send a HTTP request to the server and save the HTTP response in a response object called resp
        # 'stream = True' ensures that only response headers are downloaded initially 
        # (and not all file contents too, which are 2GB+)
        with requests.get(file_url.strip(), stream=True) as response:

            # Decode the response
            response.raw.decode_content = True
            content = response.raw

            # Write to file
            with open(download_folder / file_name, 'wb') as data:
                shutil.copyfileobj(content, data)
                
    except Exception as e:
        print('Error downloading ' + file_url + ' on try ' + str(retries_cur) + ' with error: ' + str(e))
        retries_cur += 1
        continue
    else:
        break           